In [64]:
import json

with open('export/users.json', 'r') as file:
    users = json.loads(file.read())
    users = [(user['id'], user['name']) for user in users]
    users = dict(users)

print(f'{len(users)} users found')

824 users found


In [98]:
from glob import glob
import datetime
import re

all_messages = []

def replace_user_id(match):
    id = match.group(1)
    return users.get(id, 'unknown_user')

ignored_channels = []
ignored_message_subtypes = [
    'channel_join', 
    'channel_leave', 
    'channel_purpose', 
    'file_share', 
    'channel_archive', 
    'bot_message'
]

channel_dirs = glob('export/*/')
for channel_dir in channel_dirs:
    channel_name = channel_dir.split('/')[-2]
    if channel_name in ignored_channels:
        continue
    days = os.listdir(channel_dir)
#     print(f'{len(days)} days in {channel_name}')
    for day in days:
#         print(f'Processing {channel_name}/{day}')
        with open(f'{channel_dir}{day}', 'r') as file:
            day = json.loads(file.read())
            for message in day:
                if message.get('subtype') in ignored_message_subtypes:
                    continue
                user = users.get(message.get('user'))
                    
                msg = message.get('text')
                if msg is None:
                    msg = ''

                text = re.sub(r'<@(.+)>', replace_user_id, msg)
                timestamp = datetime.datetime.fromtimestamp(int(float(message.get('ts'))))

                all_messages.append({'channel': channel_name, 'timestamp': timestamp, 'user': user, 'text': text})

print(f'{len(all_messages)} messages found')
# for msg in all_messages[:2]:
#     print(f"{msg['channel']} {msg['timestamp']} <{msg['user']}> {msg['text']}")

218871 messages found
